# (solution) Road networks structure analysis: A preliminary network science-based approach

In [ ]:
import osmnx as ox
import networkx as nx
import powerlaw

import pandas as pd

import json

from keplergl import KeplerGl
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
MAIN_CRS = 'EPSG:32633'
WORLD_CRS = 'EPSG:4326'

CITY_NAME = "Graz, Austria"

In [ ]:
# download the road network of Graz
G = ox.graph_from_place(CITY_NAME, network_type='drive', simplify= True) # a network is often represented by the letter G 
G = ox.project_graph(G, to_crs=MAIN_CRS) # this give a multigraph

G_simple = nx.DiGraph(G) # this give a simple directed graph

# print basic stats of the network
stats_json = ox.stats.basic_stats(G)
print(json.dumps(stats_json, indent=2)) # make the print output prettier 

# plot the network
fig, ax = ox.plot_graph(G, node_size=5, bgcolor= "#040615", node_color='#f1f1f1', edge_color='#d44e5c')

In [ ]:
df_centrality = pd.DataFrame.from_dict(centrality, orient='index', columns=['value'])
print("average centrality: ",(df_centrality['value'] * df_centrality.index).sum() / df_centrality['value'].sum())

fig = px.bar(df_centrality)
fig.show()

### Centralities Values

In [ ]:
# 1. extract nodes and edges
df_nodes, df_edges = ox.graph_to_gdfs(G, nodes=True)

# 2. Compute all metrics once and store them in a dictionary
metrics = {
    # "column_name": "values dict"
    "centrality":  nx.degree_centrality(G),
    "knn":         nx.average_neighbor_degree(G),
    "closeness":   nx.closeness_centrality(G),
    "betweenness": nx.betweenness_centrality(G, weight="length"), # long process
    "eigenvector": nx.eigenvector_centrality(G_simple, max_iter=1000),
    "pagerank":    nx.pagerank(G, alpha=0.9),
}

# 3. Add each metric as a new column in df_nodes
for column_name, values_dict in metrics.items():
    for metric, value in metrics.items():
        # values_dict is: {node: value} -> create a series from the dictionary with the nodes as the index.
        df_nodes[column_name] = pd.Series(values_dict)

map = KeplerGl(height=600)
map.add_data(data=df_nodes, name="nodes_Graz")
map.add_data(data=df_edges, name="edges_Graz")
map

In [ ]:
df_nodes.describe()

In [ ]:
fig = px.line(df_nodes[['centrality', 'closeness', 'betweenness', 'eigenvector']].sample(1000).reset_index(drop=True))
fig.show()

In [ ]:
# Fit the data using powerlaw, starting from minimum degree 1
fit = powerlaw.Fit(df_nodes['street_count'], xmin=1)

# Plot the  with fits (this matches the style of the provided graph)
fig = fit.plot_ccdf(color='black', label='Empirical Data')
fit.power_law.plot_ccdf(color='r', linestyle='--', ax=fig, label='Power Law')
fit.lognormal.plot_ccdf(color='g', linestyle='--', ax=fig, label='Lognormal')
fit.stretched_exponential.plot_ccdf(color='b', linestyle='--', ax=fig, label='Stretched exponential')

# Customize the plot to match the example

fig.legend(loc='lower left')
plt.show()

# To compare fits, use likelihood ratio tests
print("Power law vs Lognormal:", fit.distribution_compare('power_law', 'lognormal'))
print("Power law vs Exponential:", fit.distribution_compare('power_law', 'exponential'))
print("Power law vs Stretched exponential:", fit.distribution_compare('power_law', 'stretched_exponential'))

### Community Detection

In [ ]:
# Calculate graph initial modularity
# Defines how well a network is divided into communities range (-0.5 to 1) (higher indicates meaningful community structure)
nx.community.modularity(G.to_undirected(), nx.community.louvain_communities(G.to_undirected(), weight='length', resolution=0.07))

In [ ]:
# Louvain community detection
louvain = nx.community.louvain_communities(G.to_undirected(), weight='length', resolution=0.07) # change the resolution to find more or less communities
print('Number of communities:', len(louvain))

In [ ]:
# create a dictionary with the community id and the node id
dict_communities = {}
for community_id, nodes in enumerate(louvain):
    for node in nodes: 
        dict_communities[node] = community_id
dict_communities

# add the community id to the nodes dataframe
df_nodes["louvain_community"] = pd.Series(dict_communities) 

In [ ]:
# get the Graz city districts to overlay on the community 
admin_osm_tags = {'admin_level': '9', 'boundary_type': 'administrative'}
df_districts = ox.features_from_place(CITY_NAME, admin_osm_tags)
df_districts = df_districts.to_crs(ox.project_graph(G).graph['crs']) # project the districts to the same crs as the graph

In [ ]:
map = KeplerGl(height=800)
map.add_data(data=df_districts.loc['relation'], name="district") # admin district are recorded as relation polygons: get the relation index
map.add_data(data=df_nodes[['geometry', 'louvain_community']], name="nodes_Graz")

map

In [ ]:
# this line is to clear the output of the notebook, so that when you commit it, it is clean
!jupyter nbconvert --clear-output --inplace network_sol.ipynb